# Data Science Essentials: Data Cleaning and Feature Importance
    Eric Manner
    Math 403
    October 19, 2020
    

In [198]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')

### Problem 1

The g\_t\_results.csv file is a set of parent-reported scores on their child's Gifted and Talented tests. 
The two tests, OLSAT and NNAT, are used by NYC to determine if children are qualified for gifted programs.
The OLSAT Verbal has 16 questions for Kindergardeners and 30 questions for first and second graders.
The NNAT has 48 questions. 
Using this dataset, answer the following questions.


1) What column has the highest number of null values and what percent of its values are null? Print the answer as a tuple with (column name, percentage)

2) List the columns with have mixed types that should be numeric. Print the answer as a tuple.

3) How many third graders have scores outside the valid range for the OLSAT Verbal Score? Print the answer

4) How many data values are missing (NaN)? Print the number.

In [199]:
#load the dataset
data = pd.read_csv('g_t_results.csv', index_col=None)
#get the total number of missing values
sums_null = data.isna().sum()
#now get the column that is max
max_col = np.argmax(sums_null)
print("Most null values: ({},{})\n".format(data.columns[max_col], sums_null[max_col]))

#question 2
print("Non matching data types")
print(("OLSAT Verbal Score", "OLSAT Verbal Percentile", "NNAT Non Verbal Raw Score"))
print('\n')

#question 3
#fix the broken verbal scores
data_3 = data[data['Entering Grade Level'] == '3'].copy()
data_3['OLSAT Verbal Score'] = data_3['OLSAT Verbal Score'].apply(lambda x: str(x))
#get the ones that can be converted to int
data_valid = data_3[data_3['OLSAT Verbal Score'].str.match('^[0-9][0-9]$')].copy()
#get the number that are not numbers
invalid = len(data_3['OLSAT Verbal Score']) - len(data_valid['OLSAT Verbal Score'])
#convert to integer
data_valid['OLSAT Verbal Score'] = data_valid['OLSAT Verbal Score'].apply(lambda x: int(x))
#now get the counts that are in range
data_range = data_valid[data_valid['OLSAT Verbal Score'] > 30]
percent = invalid + len(data_range['OLSAT Verbal Score'])
print("Third grades have scores outside valid range for OLSAT: {}\n".format(percent))

#question 4
sums_null = data.isna().sum().sum()
print("Total missing data values: {}".format(sums_null))

Most null values: (School Assigned,88)

Non matching data types
('OLSAT Verbal Score', 'OLSAT Verbal Percentile', 'NNAT Non Verbal Raw Score')


Third grades have scores outside valid range for OLSAT: 1

Total missing data values: 192


### Problem 2

imdb.csv contains a small set of information about 99 movies. Clean the data set by doing the following in order: 

1) Remove duplicate rows. Print the shape of the dataframe after removing the rows.

2) Drop all rows that contain missing data. Print the shape of the dataframe after removing the rows.

3) Remove rows that have data outside valid data ranges and explain briefly how you determined your ranges for each column.

4) Identify and drop columns with three or fewer different values. Print a tuple with the names of the columns dropped.

5) Convert the titles to all lower case.

Print the first five rows of your dataframe.

In [200]:
#get the data
data = pd.read_csv('imdb.csv', index_col=None)
#remove the duplicate rows
data = data.drop_duplicates()
print("Shape after removing duplicates: {}\n".format(data.shape))

#drop all rows with missing data
data = data.dropna()
print("Shape after dropping missing data: {}\n".format(data.shape))

#remove data outside of ranges for each column

#shape for duration, it is a movie so not too short or too long (1hr - 3hrs)
data = data[data['duration'] >= 60]
data = data[data['duration'] <= 200]
#cannot make too much money or negative money
data = data[data['gross'] >= 0]
data = data[data['gross'] <= 9999999999]
#has to be a year where there was tv
data = data[data['title_year'] >= 1800]
print("Shape after setting ranges: {}\n".format(data.shape))

#remove the columns with three or fewer different values
list_names = list()
for i in data.columns:
    if len(data[i].value_counts()) <= 3:
        data = data.drop([i],axis=1)
        list_names.append(i)
print("Columns that were dropped: {}\n".format(tuple(list_names)))

#convert all titles to lower case
data['movie_title'] = data['movie_title'].str.lower()
print(data.loc[:6])

Shape after removing duplicates: (93, 13)

Shape after dropping missing data: (64, 13)

Shape after setting ranges: (58, 13)

Columns that were dropped: ('color', 'language')

       director_name  duration        gross  \
1        Shane Black       195  408992272.0   
2  Quentin Tarantino       187   54116191.0   
3   Kenneth Lonergan       186      46495.0   
4      Peter Jackson       186  258355354.0   

                                 genres                          movie_title  \
1               Action|Adventure|Sci-Fi                           iron man 3   
2  Crime|Drama|Mystery|Thriller|Western                    the hateful eight   
3                                 Drama                             margaret   
4                     Adventure|Fantasy  the hobbit: the desolation of smaug   

   title_year country       budget  imdb_score  \
1        2013     USA  200000000.0         7.2   
2        2015     USA   44000000.0         7.9   
3        2011     usa   14000000.0   

### Problem 3

basketball.csv contains data for all NBA players between 2001 and 2018.
Each row represents a player's stats for a year.

Create two new features:

    career_length (int): number of years player has been playing
    
    target (str): The target team if the player is leaving. If the player is retiring, the target should be 'retires'.


Remove all rows except those where a player changes team, that is, target is not null nor 'retires'.

Drop the player, year, and team_id columns.

Use the provided function, identify_importance(), to determine how important each feature is in a Random Forest algorithm by passing in the dataframe.
It will return a dictionary of features with the feature importance (in %) as values.
Sort the resulting dictionary from most important feature to least and print the results.

In [201]:
def identify_importance(df):
    """ Run the dataframe through a Random Forest Algorithm trying to classify the target team
    
        Parameters:
            df (dataframe): Basketball DataFrame from problem 3
            
        Returns:
            feature_import (dict): feature: importance 
    """
    y = df['target']
    X = df.drop('target',axis=1)
    forest = RandomForestClassifier(max_depth=None,min_samples_split=2)
    forest.fit(X, y)
    importances = forest.feature_importances_
    feature_import = {feature:value for feature,value in zip(X.columns,importances)}
    return feature_import

In [221]:
#get the data and drop duplicates
data = pd.read_csv('basketball.csv', index_col=0)
#get the years the players played
data = data.drop_duplicates()
year_counts = data.index.value_counts()
data['career_length'] = year_counts
data = data.reset_index()
#now create the target column
#get the year range
year_low = data['year'].value_counts().idxmin()
year_high = data['year'].value_counts().idxmax()
#loop through the range
prev_data = data[data['year']==year_low].copy()
prev_names = list(data[data['year']==year_low]['player'])
data['target'] = np.nan
for i in range(year_low+1, year_high+1):
    current_data = data[data['year']==i].copy()
    current_names = list(current_data['player'])
    #check for matches and non matches
    for name in prev_names:
        if (name in current_names):
            target_team = data.loc[list(current_data[current_data['player']==name].index)[0]]['team_id']
            past_team = data.loc[list(prev_data[prev_data['player']==name].index)[0]]['team_id']
            if (target_team == past_team):
                data = data
            else:
                data['target'].loc[list(prev_data[prev_data['player']==name].index)[0]] = target_team
        else:
            data['target'].loc[list(prev_data[prev_data['player']==name].index)[0]] = 'retires'
    prev_data = current_data
    prev_names = current_names
    
#drop all the nan values
data = data.dropna()
data = data.drop(['player','year','team_id'],axis=1)
features = identify_importance(data)
features = {k: v for k, v in sorted(features.items(), key=lambda item: item[1], reverse=True)}
print(features)

{'per': 0.2278658056716683, 'bpm': 0.2183758671815802, 'ws': 0.2034415953586269, 'career_length': 0.18339970439289516, 'age': 0.1669170273952295}


### Problem 4

Load housing.csv into a dataframe with index=0. Descriptions of the features are in housing_data_description.txt.  
The goal is to construct a regression model that predicts SalePrice using the other features of the dataset.  Do this as follows:

	1) Identify and handle the missing data.  Hint: Dropping every row with some missing data is not a good choice because it gives you an empty dataframe.  What can you do instead?
    
	2) Identify the variable with nonnumeric values that are misencoded as numbers.  One-hot encode it. Hint: don't forget to remove one of the encoded columns to prevent collinearity with the constant column (which you will add later).
    
    3) Add a constant column to the dataframe.

    4) Save a copy of the dataframe.

	5) Choose four categorical featrues that seem very important in predicting SalePrice. One-hot encode these features and remove all other categorical features.
		
	6) Run an OLS regression on your model.  

	
Print the ten features that have the highest coef in your model and the summary. 

In [301]:
#def get_top_ten(results):


#load the data
housing_data = pd.read_csv('housing.csv',index_col=0)
housing_data = housing_data.drop_duplicates()
list_to_none = ['PoolQC','Fence','MiscFeature', 'GarageCond','GarageQual','GarageFinish','GarageType','FireplaceQu','BsmtFinType1','BsmtFinType2','BsmtExposure','BsmtQual','BsmtCond','MasVnrType']
#fill the alley nans with none
housing_data['Alley'] = housing_data['Alley'].fillna('None')
#fill the lot frontage values with the average
housing_data['LotFrontage'] = housing_data['LotFrontage'].fillna(housing_data['LotFrontage'].mean())
#fill the other columns with nones when applicable
housing_data[list_to_none] = housing_data[list_to_none].fillna('None')
#fill the values with the average when applicable
housing_data['GarageYrBlt'] = housing_data['GarageYrBlt'].fillna(housing_data['GarageYrBlt'].mean())
housing_data['MasVnrArea'] = housing_data['MasVnrArea'].fillna(housing_data['MasVnrArea'].mean())
housing_data = housing_data.dropna()
#one hot encode the data column
housing_data = pd.get_dummies(housing_data, columns=['MSSubClass'], drop_first=True)
#add the constant column
housing_data = sm.add_constant(housing_data)
#save the data frame
housing_data.to_csv('housing_clean.csv',index=None)
y = housing_data['SalePrice']
X = housing_data[['const','HouseStyle','OverallCond','Foundation','BldgType']]
X = pd.get_dummies(X, columns=['HouseStyle', 'OverallCond', 'Foundation', 'BldgType'], drop_first=True)
results = sm.OLS(y,X).fit()
#print the results
print(results.params.nlargest(10))
print(results.summary())

OverallCond_9        126280.747512
OverallCond_5         82519.869661
Foundation_PConc      82296.843595
OverallCond_7         73258.759490
OverallCond_8         69789.551491
OverallCond_6         67212.975055
HouseStyle_2.5Fin     64416.110360
OverallCond_2         54284.623069
const                 53418.341257
OverallCond_4         45839.914892
dtype: float64
                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.333
Model:                            OLS   Adj. R-squared:                  0.321
Method:                 Least Squares   F-statistic:                     29.78
Date:                Mon, 19 Oct 2020   Prob (F-statistic):          4.85e-108
Time:                        20:09:19   Log-Likelihood:                -18237.
No. Observations:                1459   AIC:                         3.652e+04
Df Residuals:                    1434   BIC:                         3.666e+04
Df 

### Problem 5

Using the copy of the dataframe you created in Problem 4, one-hot encode all the categorical variables.
Print the shape of the dataframe and run OLS.

Print the ten features that have the highest coef in your model and the summary.
Write a couple of sentences discussing which model is better and why.

In [305]:
data_housing = pd.read_csv('housing_clean.csv',index_col=0)
#get the the columns to one hot encode
list_columns = list()
for it, dtype in enumerate(data_housing.dtypes):
    if (dtype == 'object'):
        list_columns.append(data_housing.columns[it])
#now create the new data with one hot encodings
data_housing = pd.get_dummies(data_housing, columns=list_columns, drop_first=True)
data_housing = sm.add_constant(data_housing)
X = data_housing.drop(['SalePrice'],axis=1)
print(X.shape)
y = data_housing['SalePrice']
#now fit the model
results = sm.OLS(y,X).fit()
#print the results
print(results.params.nlargest(10))

(1459, 273)
RoofMatl_Membran    679205.878552
RoofMatl_Metal      647138.700450
RoofMatl_WdShngl    637564.229973
RoofMatl_Tar&Grv    583341.683298
RoofMatl_CompShg    582246.144165
RoofMatl_Roll       576425.593180
RoofMatl_WdShake    573905.656952
PoolQC_None         235166.220344
GarageCond_Po       123028.599450
GarageCond_TA       121633.408062
dtype: float64


<p> I would say that the model in problem 4 is a lot more reliable. The coefficient values in problem 5 are close in value, for a lot of them. Also it took a bit longer to fit. Mostly, I would assume that 4 is better because it allows for more variety to fit and only really considers the most important features. This model I would think would be overfit. </p>